In [ ]:
import math
import numpy as np
import cmath as cm
import random
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import chainer
from chainer import configuration
from chainer.dataset import convert
import sklearn
from sklearn.utils import shuffle
from tabulate import tabulate
import datetime

#Import the module containing the class pole
import import_ipynb
import final_pole_module
from final_pole_module import detect_cusp, unif_pole, T1, T2, Einput,  generate_timestamp, seerealimagpart
from final_pole_module import mu1, mu2, T1, T2, T4, Nreal, Nimag, hbarc, phase_space, polynomial
from final_pole_module import Einput, Ereal, Eimag, Erealfar, Eimagfar, E_exp, labelz, inspect, NEpoints
from final_pole_module import skip_duplicate, export_data, import_data, get_traintest, directory

In [ ]:
def gen_ccdataset03(Ereal, Eimag, Erealfar, Eimagfar, inspect):
    #generate dataset for output label 03: '3 poles in [bt]'
    
    Emock = []
    ModEsq = []
    Re11 = []
    Re22 = []
    Im11 = []
    Im22 = []
    labelout = []
    data_info = []
    
    
    # Possible Riemann sheets
    RSpole = [[-1, 1], [-1, -1], [1, -1]] #bt  bb tb
    
    # Shuffle indexing
    for lst in [Ereal, Eimag, Erealfar, Eimagfar]:
        for _ in range(10):
            np.random.shuffle(lst)

    # Generate poles in each Riemann sheet
    for real1 in range(Nreal):
        for imag1 in range(Nimag):

            # To avoid repeated poles, we introduce random index
            indx = skip_duplicate(real1, imag1, Nreal, Nimag)
            
            
            #Generate Relevant 2-pole in [tb] sheet
            pole01 = unif_pole(RSpole[0], Ereal[indx[0][0]], Eimag[indx[1][0]] )
            pole02 = unif_pole(RSpole[1], Ereal[indx[0][1]], Eimag[indx[1][1]] )
            pole03 = unif_pole(RSpole[0], Ereal[indx[0][2]], Eimag[indx[1][2]] )
            pole04 = unif_pole(RSpole[0], Ereal[indx[0][3]], Eimag[indx[1][3]] )
            
            # Generate BG poles in different Riemann sheet
            pole1 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][4]], Eimagfar[indx[1][4]] )
            pole2 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][5]], Eimagfar[indx[1][5]] )
            pole3 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][6]], Eimagfar[indx[1][6]] )
            pole4 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][7]], Eimagfar[indx[1][7]] )
            pole5 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][8]], Eimagfar[indx[1][8]] )
            pole6 = unif_pole( RSpole[np.random.choice([1, 2])], Erealfar[indx[0][9]], Eimagfar[indx[1][9]] )
        
        pole_list_main = [pole01, pole02, pole03, pole04]
        pole_list_bg = [pole1, pole2, pole3, pole4, pole5, pole6]
        pole_list = [pole01, pole02, pole03, pole04, pole1, pole2, pole3, pole4, pole5, pole6]
        
        # Calculate BG S-matrices for each pole for all energy points
        smat11_values = np.array([pole.smat11(Einput) for pole in pole_list_main])
        smat22_values = np.array([pole.smat22(Einput) for pole in pole_list_main])
        smatdet_values = np.array([pole.smatdet(Einput) for pole in pole_list_main])

        # Calculate BG S-matrices for each pole for all energy points
        smat11_values_bg = np.array([pole.smat11(Einput) for pole in pole_list_bg])
        smat22_values_bg = np.array([pole.smat22(Einput) for pole in pole_list_bg])
        smatdet_values_bg = np.array([pole.smatdet(Einput) for pole in pole_list_bg])

        q01 = 1
        q02 = 1
        q03 = 0
        q04 = 0
        q1 = 0#np.random.choice([0, 1])
        q2 = 0#np.random.choice([0, 1])
        q3 = 0#np.random.choice([0, 1])
        q4 = 0#np.random.choice([0, 1])
        q5 = 0#np.random.choice([0, 1])
        q6 = 0#np.random.choice([0, 1])
        
        q_list = [q01, q02, q03, q04, q1, q2, q3, q4, q5, q6]

        #pwa11
        combined_smat11 = ((1-q04)+q04*smat11_values[3])*((1-q03)+q03*smat11_values[2])*((1-q02)+q02*smat11_values[1])*((1-q01)+q01*smat11_values[0])*((1-q1)+q1*smat11_values_bg[0])*((1-q2)+q2*smat11_values_bg[1])*((1-q3)+q3*smat11_values_bg[2])*((1-q4)+q4*smat11_values_bg[3])*((1-q5)+q5*smat11_values_bg[4])*((1-q6)+q6*smat11_values_bg[5])
        pwa11 = (combined_smat11 - 1.0) / (2j)
        
        #pwa22
        combined_smat22 = ((1-q04)+q04*smat22_values[3])*((1-q03)+q03*smat22_values[2])*((1-q02)+q02*smat22_values[1])*((1-q01)+q01*smat22_values[0])*((1-q1)+q1*smat22_values_bg[0])*((1-q2)+q2*smat22_values_bg[1])*((1-q3)+q3*smat22_values_bg[2])*((1-q4)+q4*smat22_values_bg[3])*((1-q5)+q5*smat22_values_bg[4])*((1-q6)+q6*smat22_values_bg[5])
        pwa22 = (combined_smat22 - 1.0) / (2j)*np.heaviside(Einput-T2*197.3,0)
        
        #pwa12
        combined_smatdet = ((1-q04)+q04*smatdet_values[3])*((1-q03)+q03*smatdet_values[2])*((1-q02)+q02*smatdet_values[1])*((1-q01)+q01*smatdet_values[0])*((1-q1)+q1*smatdet_values_bg[0])*((1-q2)+q2*smatdet_values_bg[1])*((1-q3)+q3*smatdet_values_bg[2])*((1-q4)+q4*smatdet_values_bg[3])*((1-q5)+q5*smatdet_values_bg[4])*((1-q6)+q6*smatdet_values_bg[5])
        pwa12sqr = ((-1.0)*((combined_smat11*combined_smat22-combined_smatdet))/4.0)*np.heaviside(Einput-T2*197.3,0)
        mod12sqr = np.abs(pwa12sqr)
        arg12sqr = np.unwrap(np.array([np.angle(z) for z in pwa12sqr]))
        pwa12 = np.sqrt(mod12sqr)*np.exp(1j* arg12sqr / 2)

        # Calculate PWAT
        pwatadd = pwa11 + pwa12
        PWAT = np.abs(pwatadd)**2 #np.conjugate(pwa11) * pwa11 + np.conjugate(pwa12) * pwa12 + np.conjugate(pwa11)*pwa12 +pwa11*np.conjugate(pwa12)

        coeff = np.random.uniform(0, 5, 3)
        A = np.random.uniform(1, 1000)
        poly_bg = polynomial(coeff, Einput)
        
        sim_exp = A*phase_space*(PWAT + poly_bg)
        PWAT = sim_exp
        
        # Collect input data
        Emock.append(Einput) 
        ModEsq.append(PWAT)
        Re11.append(pwa11.real)
        Im11.append(pwa11.imag)
        Re22.append(pwa22.real)
        Im22.append(pwa22.imag)
        labelout.append(3)


        if inspect == True:
            information = [[pole.pos for pole in pole_list], 
                           [pole.RS for pole in pole_list],
                           q_list
                          ]
            data_info.append(information)

    return Emock, ModEsq, Re11, Im11, Re22, Im22, labelout, data_info

In [ ]:
Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03  = gen_ccdataset03(Ereal, Eimag, Erealfar, Eimagfar, inspect)
out = directory
pickle.dump (Einput03, open(os.path.join(out,'Einput03.pkl'),'wb'), protocol=4)
pickle.dump (ModEsq03, open(os.path.join(out,'ModEsq03.pkl'),'wb'), protocol=4)
pickle.dump (labelout03, open(os.path.join(out,'labelout03.pkl'),'wb'), protocol=4)
pickle.dump (data_info03, open(os.path.join(out,'data_info03.pkl'),'wb'), protocol=4)
#The following will collect the data that will go to the input layer:
#You can design a DNN with Einput and T11sqr in the input layer
# PWATsqr00 = [x*np.conjugate(x) + y*np.conjugate(y) + np.conjugate(x)*y + x*np.conjugate(y) for x, y in zip(RePWAT00, ImPWAT00)]
PWATsqr03 = np.concatenate((Einput03,ModEsq03), axis=1)
#or you can design a DNN with Einput, ReT11 and ImT11 in the input layer
# PWAT00 = np.concatenate((Einput00,ReT1100, ImT1100), axis=1)
pickle.dump (PWATsqr03, open(os.path.join(out,'PWATsqr03.pkl'),'wb'), protocol=4)
# pickle.dump (T1100, open(os.path.join(out,'T1100.pkl'),'wb'), protocol=4)
# del Einput21, ModEsq21, RePWAT21, ImPWAT21, labelout21, data_info21, PWATsqr21

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
seerealimagpart(Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03)

In [ ]:
del Einput03, ModEsq03, Re11_03, Im11_03, Re22_03, Im22_03, labelout03, data_info03